In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm, nbinom, gamma
from datetime import timedelta
import psycopg2

In [2]:
df = pd.read_csv(
    "acompanhamento_suitability(Planilha1).csv",
    sep=";",
    encoding="latin1"
)

# limpar nomes das colunas logo no começo
df.columns = df.columns.astype(str).str.strip()

In [3]:
# remover colunas Unnamed e coluna vazia (por causa do ;;)
df = df.loc[:, ~df.columns.str.contains("Unnamed", na=False)]
df = df.loc[:, df.columns != ""]  # remove coluna vazia do ;; se existir

print(df.head())
print(df.columns)

  Cliente  grupo        tipo  tamanho  desconto  skus_sem1  skus_sem2  \
0  000070    NaN      Varejo   Grande       NaN        NaN        NaN   
1  000080    NaN      Varejo   Grande       NaN        NaN        NaN   
2  000346    NaN      Varejo   Grande       NaN        NaN        NaN   
3  000396    NaN  Autocentro  Pequeno       NaN        NaN        NaN   
4  000415    NaN      Varejo  Pequeno       NaN        NaN        NaN   

   skus_sem3  fat_sem1  fat_sem2  fat_sem3  marg_sem1  marg_sem2  marg_sem3  
0        NaN       NaN       NaN       NaN        NaN        NaN        NaN  
1        NaN       NaN       NaN       NaN        NaN        NaN        NaN  
2        NaN       NaN       NaN       NaN        NaN        NaN        NaN  
3        NaN       NaN       NaN       NaN        NaN        NaN        NaN  
4        NaN       NaN       NaN       NaN        NaN        NaN        NaN  
Index(['Cliente', 'grupo', 'tipo', 'tamanho', 'desconto', 'skus_sem1',
       'skus_sem2', 's

In [4]:
# garantir tipo e tamanho limpos
df["tipo"] = df["tipo"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
df["tamanho"] = df["tamanho"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()

# criar classificação (tipo + tamanho)
df["classificacao"] = (df["tipo"] + " " + df["tamanho"]).astype(str)
df["classificacao"] = df["classificacao"].str.replace(r"\s+", " ", regex=True).str.strip()

# remover classificações inválidas ANTES de tudo (não pode ir pro CSV final)
padroes_invalidos = r"(?i)^não qualificado\b|^#n/d\b|^falso\s+falso\b"

In [5]:

antes = df.shape[0]
df = df[~df["classificacao"].str.contains(padroes_invalidos, regex=True, na=False)].copy()
df = df[~df["tipo"].str.contains(padroes_invalidos, regex=True, na=False)].copy()
df = df[~df["tamanho"].str.contains(padroes_invalidos, regex=True, na=False)].copy()
depois = df.shape[0]

print(f"Linhas removidas por classificação inválida: {antes - depois}")
print("Classificações restantes:")
print(df["classificacao"].value_counts().head(20))

Linhas removidas por classificação inválida: 100
Classificações restantes:
classificacao
Autocentro Pequeno    905
Autocentro Grande     399
Varejo Grande         283
Varejo Pequeno        273
nan nan               193
Name: count, dtype: int64


In [6]:
if "Unnamed: 2" in df.columns:
    df = df.rename(columns={"Unnamed: 2": "procura"})

# %%
# contagem por classificacao
contagem = df["classificacao"].value_counts()
percentual = df["classificacao"].value_counts(normalize=True) * 100

print("\n=== Quantidade por classificação ===")
print(contagem)


=== Quantidade por classificação ===
classificacao
Autocentro Pequeno    905
Autocentro Grande     399
Varejo Grande         283
Varejo Pequeno        273
nan nan               193
Name: count, dtype: int64


In [7]:

print("\n=== Percentual por classificação ===")
print(percentual)



=== Percentual por classificação ===
classificacao
Autocentro Pequeno    44.081831
Autocentro Grande     19.434973
Varejo Grande         13.784705
Varejo Pequeno        13.297613
nan nan                9.400877
Name: proportion, dtype: float64


In [8]:
conn_params = {
    "host": "srvdados",
    "database": "postgres",
    "user": "compras",
    "password": "pecist@compr@s2024"
}

cli = '''
select
  pp.codcli,
  sum(pp.preco) as preco,
  sum(pp.qtde_ven) as qtde,
  sum(pp.qtde_ven * pp.preco) as faturamento
from "D-1".prod_ped pp
join "D-1".cliente cli on pp.codcli = cli.codcli
where pp.cd_loja = '08' and pp.dt_emissao >= '2025-10-01' and pp.dt_emissao < '2025-12-31' and cli.codarea != '112' and cli.sigladesc not in ('O','Q','P')
group by pp.codcli
'''

with psycopg2.connect(**conn_params) as conn:
    clientes = pd.read_sql(cli, conn)


C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_22860\3709342717.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  clientes = pd.read_sql(cli, conn)


In [9]:
clientes["codcli"] = clientes["codcli"].astype(str).str.strip()
clientes["faturamento"] = pd.to_numeric(clientes["faturamento"], errors="coerce").fillna(0)

# remover faturamento 0 na query
antes = clientes["codcli"].nunique()
clientes = clientes[clientes["faturamento"] > 0].copy()
depois = clientes["codcli"].nunique()
print(f"Clientes removidos por faturamento zero (na query): {antes - depois}")

Clientes removidos por faturamento zero (na query): 0


In [10]:
clientes = clientes.rename(columns={"codcli": "Cliente"})
clientes_merge = clientes[["Cliente", "faturamento", "qtde", "preco"]].copy()

In [11]:
# merge faturamento no df
df = df.merge(clientes_merge, on="Cliente", how="left")
df["faturamento"] = df["faturamento"].fillna(0)

# remover do df clientes sem faturamento (0 ou ausente)
antes_df = df["Cliente"].nunique()
clientes_ativos = df.loc[df["faturamento"] > 0, "Cliente"].drop_duplicates()
df = df[df["Cliente"].isin(clientes_ativos)].copy()
depois_df = df["Cliente"].nunique()

In [12]:
print(f"Clientes removidos do DF por faturamento zero/ausente na query: {antes_df - depois_df}")
print("Clientes restantes no DF:", depois_df)

print(df[["Cliente", "classificacao", "faturamento"]].head())

Clientes removidos do DF por faturamento zero/ausente na query: 1161
Clientes restantes no DF: 699
    Cliente       classificacao  faturamento
399   I0100   Autocentro Grande     63040.91
402   I0836  Autocentro Pequeno      9750.11
403   I0909   Autocentro Grande     14753.41
407   I1520   Autocentro Grande     38548.24
412   I1536   Autocentro Grande      2163.98


In [13]:
# embaralhar AGORA (depois do filtro)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# %%
# base única por cliente+classificacao (pra não duplicar cliente no sorteio)
base_clientes = (
    df[["Cliente", "classificacao"]]
    .drop_duplicates()
    .sample(frac=1, random_state=42)
    .reset_index(drop=True)
)

In [14]:
grupos = []

for classe, sub in base_clientes.groupby("classificacao"):
    sub = sub.sample(frac=1, random_state=42).reset_index(drop=True)

    n = len(sub)
    tamanho = n // 3
    sobra = n % 3

    gA = sub.iloc[0:tamanho]
    gB = sub.iloc[tamanho:2*tamanho]
    gC = sub.iloc[2*tamanho:3*tamanho]

    if sobra >= 1:
        gA = pd.concat([gA, sub.iloc[3*tamanho:3*tamanho+1]])
    if sobra >= 2:
        gB = pd.concat([gB, sub.iloc[3*tamanho+1:3*tamanho+2]])
    if sobra == 3:
        gC = pd.concat([gC, sub.iloc[3*tamanho+2:3*tamanho+3]])

    gA = gA.assign(grupo="A")
    gB = gB.assign(grupo="B")
    gC = gC.assign(grupo="C")

    grupos.extend([gA, gB, gC])

df_grupos = pd.concat(grupos).reset_index(drop=True)

print("\n=== Qtd de clientes por grupo ===")
print(df_grupos["grupo"].value_counts())



=== Qtd de clientes por grupo ===
grupo
A    233
B    233
C    233
Name: count, dtype: int64


In [15]:
print("\n=== Qtd de clientes por grupo e classificação ===")
print(df_grupos.groupby(["grupo", "classificacao"]).size())


=== Qtd de clientes por grupo e classificação ===
grupo  classificacao     
A      Autocentro Grande      50
       Autocentro Pequeno    119
       Varejo Grande          24
       Varejo Pequeno         40
B      Autocentro Grande      50
       Autocentro Pequeno    119
       Varejo Grande          24
       Varejo Pequeno         40
C      Autocentro Grande      50
       Autocentro Pequeno    119
       Varejo Grande          24
       Varejo Pequeno         40
dtype: int64


In [16]:
# se o CSV já tem coluna 'grupo', remove antes do merge pra não virar grupo_x/grupo_y
if "grupo" in df.columns:
    df = df.drop(columns=["grupo"])

resultado = df.merge(
    df_grupos[["Cliente", "classificacao", "grupo"]],
    on=["Cliente", "classificacao"],
    how="left"
)

In [17]:
# garantir ID_ordem (se por algum motivo não existir mais)
if "ID_ordem" not in resultado.columns:
    # tenta herdar do df (se existir lá)
    if "ID_ordem" in df.columns:
        resultado = resultado.merge(
            df[["Cliente", "classificacao", "ID_ordem"]].drop_duplicates(),
            on=["Cliente", "classificacao"],
            how="left"
        )
    else:
        # último recurso: cria uma ordem atual
        resultado["ID_ordem"] = range(len(resultado))

resultado = resultado.sort_values("ID_ordem").reset_index(drop=True)

print("\nChecagem resultado:")
print(resultado[["Cliente", "classificacao", "grupo", "faturamento"]].head())

print("\nLinhas sem grupo:", resultado["grupo"].isna().sum())


Checagem resultado:
  Cliente       classificacao grupo  faturamento
0   I4582  Autocentro Pequeno     B      2545.50
1   I3998   Autocentro Grande     B     80008.23
2   I1624   Autocentro Grande     C      3451.47
3   I4458  Autocentro Pequeno     C      1787.12
4   I6437      Varejo Pequeno     C      2001.49

Linhas sem grupo: 0


In [18]:
base_resumo = resultado[["Cliente", "classificacao", "grupo", "faturamento"]].drop_duplicates(
    subset=["Cliente", "classificacao", "grupo"]
)

resumo = (
    base_resumo
    .groupby(["classificacao", "grupo"])
    .agg(
        qtd_clientes=("Cliente", "nunique"),
        faturamento_total=("faturamento", "sum")
    )
    .reset_index()
)


# pivots
qtd_pivot = resumo.pivot(index="classificacao", columns="grupo", values="qtd_clientes").fillna(0).astype(int)
fat_pivot = resumo.pivot(index="classificacao", columns="grupo", values="faturamento_total").fillna(0)

# organizar bonitinho em pivot (lado a lado)
qtd_pivot = qtd_pivot.reindex(columns=["A", "B", "C"], fill_value=0)
fat_pivot = fat_pivot.reindex(columns=["A", "B", "C"], fill_value=0)


In [19]:
for classe in qtd_pivot.index:
    print(f"\n{classe}")
    for g in ["A", "B", "C"]:
        qtd = int(qtd_pivot.loc[classe, g])
        fat = float(fat_pivot.loc[classe, g])
        print(f"  Grupo {g}: {qtd} clientes | faturamento {fat:,.2f}")


Autocentro Grande
  Grupo A: 50 clientes | faturamento 604,202.18
  Grupo B: 50 clientes | faturamento 811,369.69
  Grupo C: 50 clientes | faturamento 561,424.23

Autocentro Pequeno
  Grupo A: 119 clientes | faturamento 957,817.41
  Grupo B: 119 clientes | faturamento 1,009,366.65
  Grupo C: 119 clientes | faturamento 906,835.58

Varejo Grande
  Grupo A: 24 clientes | faturamento 182,093.71
  Grupo B: 24 clientes | faturamento 393,466.09
  Grupo C: 24 clientes | faturamento 209,363.90

Varejo Pequeno
  Grupo A: 40 clientes | faturamento 430,221.85
  Grupo B: 40 clientes | faturamento 354,477.93
  Grupo C: 40 clientes | faturamento 289,384.30


In [ ]:
#resultado.to_excel("classificacao_final.xlsx",index=False)